In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

Empresas Programas por empresa y empresas desde account

In [2]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(7924, 7)

In [3]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7928, 5)

In [4]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas.shape

(7932, 12)

In [5]:
Programas_pacientes = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_pacientes = Programas_pacientes[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]
Programas_pacientes.head()

,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Programas_por_Empresas__c,Codigo EMP,Codigo A
0,0018c00002eTcCQAA0,Vida Camara Holding Vida Camara Cintegral Sist...,Holding Vida Camara,Vida Camara,None,None,a1W8c0000083l2BEAQ,EMP879,A-2357
1,0018c00002eTcCRAA0,Vida Camara Holding Vida Camara Civil Agro S.A.,Holding Vida Camara,Vida Camara,None,None,a1W8c0000083l2CEAQ,EMP891,A-2358
2,0018c00002eTcCSAA0,Vida Camara Holding Vida Camara Civilplas S.A.,Holding Vida Camara,Vida Camara,None,None,a1W8c0000083l2DEAQ,EMP893,A-2359
3,0018c00002eTcCTAA0,Vida Camara Holding Vida Camara Cj Traffic Chi...,Holding Vida Camara,Vida Camara,None,None,a1W8c0000083l2EEAQ,EMP894,A-2360
4,0018c00002eTcCUAA0,Vida Camara Holding Vida Camara Clinda Ltda.,Holding Vida Camara,Vida Camara,None,None,a1W8c0000083l2FEAQ,EMP895,A-2361


fin empresas y comienzo de rescate de pacientes


In [6]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1554481, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [7]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

(1268269, 4)

Union de pacientes programas con las empresas

In [8]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_pacientes,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1268269, 12)

In [9]:
empresas_unidas.head()


,Id,Paciente__c,Programas_por_Empresas__c,padre__c,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Metlife Holding Metlife Serv. De Comerc Y Ases...,MetLife,MetLife,None,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353


In [10]:
empresas_unidas = pd.merge(
    left=empresas_unidas,
    right=Pacientes_SF,
    how="left",
    left_on="Paciente__c",
    right_on="Id Account Paciente",
)
empresas_unidas.shape

(1268269, 21)

In [11]:
empresas_unidas.head()

,Id,Paciente__c,Programas_por_Empresas__c,padre__c,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,...,Codigo A,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Metlife Holding Metlife Serv. De Comerc Y Ases...,MetLife,MetLife,None,None,...,A-0173,0018c00002amLbpAAE,224488556,Genesis Antonia Gutierrez Abarca,RUT,None,None,None,Mujer,967786319
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,...,A-0353,0018c00002amLbqAAE,113376066,Claudia Silva,RUT,None,None,None,Mujer,None
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,...,A-0353,0018c00002amLbrAAE,79476161,Maria Lorenza Garay,RUT,None,None,None,Mujer,None
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,...,A-0353,0018c00002amLbsAAE,181854227,Estolaza Vidal Jean Patrick Moises,RUT,None,None,None,Hombre,953757757
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,...,A-0353,0018c00002amLc2AAE,126899823,Rivas Correa Claudia,RUT,None,None,None,Mujer,982058845


In [12]:
empresas_unidas = empresas_unidas.rename(
    columns={
        "Id": "Id Pacientes Programas"       
    }
)
empresas_unidas = empresas_unidas[
    [
        "Id Pacientes Programas",
        "Paciente__c",
        "Programas_por_Empresas__c",
        'Id Account Empresa',
        'Nombre Empresa',
        'Holding',
        'Campaña',
        'Empleador',
        'Poliza',
        'Id Account Paciente',
        'Rut',
        'Nombre',
        'Ocupacion',
        'Cargo',
        'Departamento',
        'Sexo',
        'Telefono',
        'Tipo de Documento',
        'Codigo EMP',
        'Codigo A'


    ]]
empresas_unidas.head()

,Id Pacientes Programas,Paciente__c,Programas_por_Empresas__c,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Id Account Paciente,Rut,Nombre,Ocupacion,Cargo,Departamento,Sexo,Telefono,Tipo de Documento,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,0018c00002axQaeAAE,Metlife Holding Metlife Serv. De Comerc Y Ases...,MetLife,MetLife,None,None,0018c00002amLbpAAE,224488556,Genesis Antonia Gutierrez Abarca,None,None,None,Mujer,967786319,RUT,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbqAAE,113376066,Claudia Silva,None,None,None,Mujer,None,RUT,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbrAAE,79476161,Maria Lorenza Garay,None,None,None,Mujer,None,RUT,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbsAAE,181854227,Estolaza Vidal Jean Patrick Moises,None,None,None,Hombre,953757757,RUT,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLc2AAE,126899823,Rivas Correa Claudia,None,None,None,Mujer,982058845,RUT,EMP6274,A-0353


Terminado la visualizacion de las empresas y pacientes que se muestran en los reportes


Descarga desde Citas de Servicios

In [13]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,Tipo_de_la_cita__c from ServiceAppointment
    """
)


Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio['Empresas'] = np.nan
#Citas_de_servicio['RUT__c'].map(pacientessf_empresas.set_index('IdentificationId__pc_x',verify_integrity=False)['Name']) 
Citas_de_servicio['Empresas Id'] = np.nan
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',        
        "AccountId",
        'Clinica__c'
        
    ]
]
Citas_de_servicio.shape


(7889, 8)

In [14]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c,AppointmentNumber from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2['Clinica__c'] = 'Orientación Médica'
Citas_de_servicio2['Empresas'] = np.nan
Citas_de_servicio2['Empresas Id'] = np.nan
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',       
        "AccountId",
        'Clinica__c',
        
    ]
]
Citas_de_servicio2.shape


(7889, 8)

In [15]:
#unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)
Gran_Citas_deservicio = Gran_Citas_deservicio.rename(
    columns={
        "AppointmentNumber": "Codigo Ficha",
        "RUT__c": "Rut",
        "Empresas": "Empleador",
        "CreatedDate": "Fecha",
        "Tipo_de_la_cita__c": "Canal",
        "Empresas Id": "Cod. Empresa",
        "AccountId": "Id Paciente",        
        'Clinica__c':'Clínica'
    }
)

Gran_Citas_deservicio.shape

(15778, 8)

In [16]:
Gran_Citas_deservicio.head()

,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa,Id Paciente,Clínica
0,SA-3124,183552252,NaN,2022-12-02T16:23:05.000+0000,Phone,NaN,0018c00002azpJ4AAI,Clínica de Nutrición
1,SA-8295,12654586K,NaN,2023-01-30T12:32:08.000+0000,Phone,NaN,0018c00002amTlVAAU,Clínica de Deporte
2,SA-4987,224488556,NaN,2022-12-21T21:28:49.000+0000,Phone,NaN,0018c00002amLbpAAE,Clínica de Nutrición
3,SA-3905,185384799,NaN,2022-12-12T15:34:28.000+0000,Phone,NaN,0018c00002ay6KFAAY,Clínica de Deporte
4,SA-3871,187821444,NaN,2022-12-12T12:21:02.000+0000,Phone,NaN,0018c00002bI5vJAAS,Clínica de Psicología


In [17]:
#Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_deporte['clinica'] = 'Deporte'
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_deporte.shape


(872, 8)

In [18]:
# Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])

ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "RUT__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_nutricion["clinica"] = "Nutrición"

ficha_nutricion = ficha_nutricion.rename(
    columns={
        "Name": "Codigo Ficha",
        "RUT__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)


ficha_nutricion.shape

(1975, 8)

In [19]:
#Clinica AppSex
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])

ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Apssex['clinica'] = 'Asesoría Psicologíca en Salud Sexual'
ficha_Apssex = ficha_Apssex.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)

ficha_Apssex.shape

(3, 8)

In [20]:
# Clinica Psicologia
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,Motivo_de_consulta_manifiesto__c from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])

ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "Motivo_de_consulta_manifiesto__c",
    ]
]
ficha_Psicologia["clinica"] = "Psicología"
ficha_Psicologia = ficha_Psicologia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "Motivo_de_consulta_manifiesto__c": "Motivo de consulta",
    }
)
ficha_Psicologia.shape

(1430, 9)

In [21]:
#Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Sueno['clinica'] = 'Clínica Del Sueño'
ficha_Sueno = ficha_Sueno.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Sueno.shape

(201, 8)

In [22]:
#Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Cronico['clinica'] = 'Pacientes Crónicos'
ficha_Cronico = ficha_Cronico.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Cronico.shape

(785, 8)

In [23]:
#Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Telemedicina['clinica'] = 'Telemedicina'
ficha_Telemedicina = ficha_Telemedicina.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_Telemedicina.shape

(2432, 8)

In [24]:
#Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Veterinaria['clinica'] = 'Veterinaria'
ficha_Veterinaria = ficha_Veterinaria.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Veterinaria.shape

(12, 8)

In [25]:
#Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Orientaci_n_Medica__c     
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Orientacion['clinica'] = 'Orientación Médica'
ficha_Orientacion = ficha_Orientacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Orientacion.shape

(26, 8)

In [26]:
#Clinica Front
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Front__c     
     """
)
ficha_Front = pd.DataFrame(results["records"])

ficha_Front = ficha_Front[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Front['clinica'] = 'Orientación Médica'
ficha_Front = ficha_Front.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Front.shape

(213, 8)

In [27]:
#Clinica_general
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c,Account__c,   Programa__c   from Ficha_Clinica_General__c 
     """
)
ficha_general = pd.DataFrame(results["records"])

ficha_general = ficha_general[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'Programa__c'
        
    ]
]

ficha_general = ficha_general.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'Programa__c':'Clínica'
    }
)
ficha_general.shape


(970, 8)

In [28]:
#Clinica Ingreso deportivo
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_Ingreso_dep = pd.DataFrame(results["records"])

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Ingreso_dep['clinica'] = 'Ingreso Deporte Seguro'
ficha_Ingreso_dep = ficha_Ingreso_dep.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Ingreso_dep.shape

(219, 8)

In [29]:
Clinicas = pd.concat([ficha_Front, ficha_Ingreso_dep,ficha_Orientacion,ficha_Veterinaria,ficha_Telemedicina,ficha_Cronico,ficha_Sueno,ficha_Psicologia,ficha_Apssex,ficha_nutricion,ficha_deporte,ficha_general], axis=0)
Clinicas.shape

(9138, 9)

In [30]:
Reortes_general = pd.concat([Clinicas,Gran_Citas_deservicio ], axis=0)
Reortes_general.shape

(24916, 9)

In [31]:
Reortes_general["Dia"] = pd.to_datetime(
    Reortes_general["Fecha"]
).dt.day

Reortes_general["Mes"] = pd.to_datetime(
    Reortes_general["Fecha"]
).dt.month

Reortes_general["Año"] = pd.to_datetime(
    Reortes_general["Fecha"]
).dt.year


# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Reortes_general["Fecha_chilena"] = pd.to_datetime(
    Reortes_general["Fecha"], dayfirst=True, yearfirst=False, exact=True
)
Reortes_general["Fecha_chilena2"] = Reortes_general["Fecha_chilena"].dt.strftime("%d-%m-%Y")

clinicas_ = {
    "Clínica de Nutrición": "Nutrición",
    "Clínica de Deporte": "Deporte",
    "Clínica de Psicología": "Psicología",
    "Clínica Ingreso Deportivo Seguro": "Deporte",
    "Clínica del Sueño": "Clínica Del Sueño",
    "Clínica Seguimiento Pacientes Crónicos": "Pacientes Crónicos",
    'Orientación Médica':'Orientación Médica' ,
    'Asesoría Deportiva':'Deporte',
    'Asistencia Emocional':"Psicología",
    'Asistencia Nutricional':'Nutrición',
    'Clínica APSSex':'Asesoría Psicologíca en Salud Sexual',
    'Clínica Del Sueño':'Clínica Del Sueño',
    'Clínica Veterinaria':'Veterinaria',
    'Ingreso Deportivo Seguro':'Deporte',
    'Clínica Mis Primeros Pasos':'Mis Primeros Pasos'
    }
Reortes_general.replace({"Clínica": clinicas_}, inplace=True)

Canal = {
    'Phone':'Teléfono',
    'Video':'Videollamada',
    'Entrante - Chat':'Entrante - Chat Online'
}
Reortes_general.replace({"Canal": Canal}, inplace=True)




Reortes_general['Fecha_chilena'] = Reortes_general['Fecha_chilena'].dt.tz_localize(None)
Reortes_general.Rut.fillna('0', inplace=True)




Reortes_general.head()



,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa,Id Paciente,Clínica,Motivo de consulta,Dia,Mes,Año,Fecha_chilena,Fecha_chilena2
0,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,Orientación Médica,NaN,24,1,2023,2023-01-24 18:31:33,24-01-2023
1,FFF-0154,166090636,EY - EY Audit SpA,2023-01-30T18:24:32.000+0000,Entrante - Chat Online,a1Y8c00000DxPM9EAN,0018c00002amSFbAAM,Orientación Médica,NaN,30,1,2023,2023-01-30 18:24:32,30-01-2023
2,FFF-0088,164841448,Evaluaserve Chile S.A.,2023-01-26T14:25:58.000+0000,Entrante - Chat Online,a1Y8c00000Clq5uEAB,0018c00002b03a0AAA,Orientación Médica,NaN,26,1,2023,2023-01-26 14:25:58,26-01-2023
3,FFF-0096,188075606,None,2023-01-26T17:29:57.000+0000,Entrante - Llamada,None,0018c00002eimgBAAQ,Orientación Médica,NaN,26,1,2023,2023-01-26 17:29:57,26-01-2023
4,FFF-0076,195034931,Metlife Holding Metlife Sacyr Operacion Y Serv...,2023-01-26T12:17:12.000+0000,Entrante - Chat Online,a1Y8c00000CltMaEAJ,0018c00002b0aVyAAI,Orientación Médica,NaN,26,1,2023,2023-01-26 12:17:12,26-01-2023


In [32]:

empresas_unidas.head()

,Id Pacientes Programas,Paciente__c,Programas_por_Empresas__c,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Id Account Paciente,Rut,Nombre,Ocupacion,Cargo,Departamento,Sexo,Telefono,Tipo de Documento,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,0018c00002axQaeAAE,Metlife Holding Metlife Serv. De Comerc Y Ases...,MetLife,MetLife,None,None,0018c00002amLbpAAE,224488556,Genesis Antonia Gutierrez Abarca,None,None,None,Mujer,967786319,RUT,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbqAAE,113376066,Claudia Silva,None,None,None,Mujer,None,RUT,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbrAAE,79476161,Maria Lorenza Garay,None,None,None,Mujer,None,RUT,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLbsAAE,181854227,Estolaza Vidal Jean Patrick Moises,None,None,None,Hombre,953757757,RUT,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,0018c00002amLc2AAE,126899823,Rivas Correa Claudia,None,None,None,Mujer,982058845,RUT,EMP6274,A-0353


In [33]:
Reportes_con_empresas = pd.merge(
    left=Reortes_general,
    right=empresas_unidas,
    how="left",
    left_on="Id Paciente",
    right_on="Id Account Paciente",
)


Reportes_con_empresas.shape

(27980, 34)

In [34]:
Reportes_con_empresas.head()

,Codigo Ficha,Rut_x,Empleador_x,Fecha,Canal,Cod. Empresa,Id Paciente,Clínica,Motivo de consulta,Dia,...,Rut_y,Nombre,Ocupacion,Cargo,Departamento,Sexo,Telefono,Tipo de Documento,Codigo EMP,Codigo A
0,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,Orientación Médica,NaN,24,...,181523778,Marjorie Melanie Rojas Cea,None,None,None,Mujer,956805279,RUT,EMP6475,A-2321
1,FFF-0154,166090636,EY - EY Audit SpA,2023-01-30T18:24:32.000+0000,Entrante - Chat Online,a1Y8c00000DxPM9EAN,0018c00002amSFbAAM,Orientación Médica,NaN,30,...,166090636,Rominna Paz Gaete Stagno,None,Manager,Assurance,Mujer,982680071,RUT,EMP4888,A-4489
2,FFF-0088,164841448,Evaluaserve Chile S.A.,2023-01-26T14:25:58.000+0000,Entrante - Chat Online,a1Y8c00000Clq5uEAB,0018c00002b03a0AAA,Orientación Médica,NaN,26,...,164841448,Maria Sofia Eberl Muñoz,None,None,None,Mujer,None,RUT,EMP5716,A-0341
3,FFF-0096,188075606,None,2023-01-26T17:29:57.000+0000,Entrante - Llamada,None,0018c00002eimgBAAQ,Orientación Médica,NaN,26,...,188075606,SATCHARY CARMONA GIACAMAN,None,None,None,Hombre,None,RUT,EMP6471,A-8046
4,FFF-0076,195034931,Metlife Holding Metlife Sacyr Operacion Y Serv...,2023-01-26T12:17:12.000+0000,Entrante - Chat Online,a1Y8c00000CltMaEAJ,0018c00002b0aVyAAI,Orientación Médica,NaN,26,...,195034931,Cristian Alexis Droguett Araya,None,None,None,Hombre,968355428,RUT,EMP5897,A-0084


In [35]:
Reportes_con_empresas.Empleador_x.fillna(Reportes_con_empresas['Nombre Empresa'], inplace=True)

Salida de datos

In [36]:
Reportes_con_empresas.to_excel(
    "C:/Users/jmartinez/Documents/reportes_completos.xlsx", index=False
)

Reortes_general.to_excel(
    "C:/Users/jmartinez/Documents/reportes.xlsx", index=False
)

Programas_pacientes.to_excel(
    "C:/Users/jmartinez/Documents/Empresas.xlsx", index=False
)


In [ ]:
#Reportes_con_empresas.Empleador_x.fillna(Reportes_con_empresas['Nombre Empresa'], inplace=True)
reportes_francisca = Reportes_con_empresas
reportes_francisca.Empleador_x.fillna('Nones',inplace=True)
reportes_francisca = reportes_francisca[
    reportes_francisca['Empleador_x'] == 'Nones'
]
archivo = 'SinEmpresas.xlsx'
archivo1 = 'Empresas.xlsx'

fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
#archivo_con_fecha = archivo_con_fecha.replace(".", "_")
hasta = Path("E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/Pacientes Sin Empresas/").joinpath(archivo_con_fecha)
print (hasta)
reportes_francisca.to_excel(
    hasta, index=False
)



In [40]:
fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo1}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
hasta = Path("E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/Empresas/").joinpath(archivo_con_fecha)
Programas_pacientes.to_excel(
    hasta, index=False
)

hasta = Path("F:/Empresas SF/empresas/").joinpath(archivo_con_fecha)
Programas_pacientes.to_excel(
    hasta, index=False
)